In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [5]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/февраль/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['feb.xlsx', 'jan.xlsx']
sheet = 'Sheet'

In [2]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [3]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [9]:
zp = pd.read_excel(path_to_data + 'ДОНМ_Начисления работников.xlsx')

In [27]:
zp = zp[['СНИЛС', 'Организация', 'КоличествоСтавокНаКонецМес']].rename(columns={'СНИЛС': 'snils', 'Организация': 'org', 'КоличествоСтавокНаКонецМес': 'stv'})
zp

,snils,org,stv
0,001-879-913 57,ГБПОУ СКИСиГ,"1,00000000000000000000"
1,001-896-420 39,ГБПОУ СКИСиГ,"1,00000000000000000000"
2,002-050-383 67,ГБПОУ СКИСиГ,"1,00000000000000000000"
3,002-065-271 85,ГБПОУ СКИСиГ,"1,00000000000000000000"
4,002-845-525 25,ГБПОУ СКИСиГ,"1,00000000000000000000"
...,...,...,...
199468,186-373-917 14,ГБОУ Школа «Содружество»,"1,00000000000000000000"
199469,187-703-552 01,ГБОУ Школа «Содружество»,"1,00000000000000000000"
199470,198-391-256 24,ГБОУ Школа «Содружество»,"1,00000000000000000000"
199471,202-693-820 52,ГБОУ Школа «Содружество»,"1,00000000000000000000"


In [32]:
zp['stv'] = zp.apply(lambda row: to_double(row, 'stv'), axis=1)

In [12]:
kadr = pd.read_excel(path_to_data + 'ДОНМ_Кадровые_паспортные данные.xlsx')

In [15]:
kadr = kadr[['СНИЛС', 'ДР']]
kadr

,СНИЛС,ДР
0,170-535-764 66,11.02.1991
1,170-535-764 66,11.02.1991
2,164-298-981 16,10.05.1982
3,159-429-243 98,05.01.1998
4,054-243-407 30,25.07.1972
...,...,...
217693,016-022-613 91,07.03.1967
217694,176-492-809 15,14.08.1998
217695,151-620-104 08,15.08.1986
217696,113-021-844 88,25.01.1972


In [22]:
def fil(row):
    t = str(row['ДР']).split('.')
    t = list(map(int, t))
    if ((t[2] > 2002) & (t[2] < 2008)) | ((t[2] == 2002) & (t[1] >= 4)) | ((t[2] == 2008) & (t[1] <= 4)) :
        return 1
    return 0

In [23]:
kadr['ans'] = kadr.apply(lambda row: fil(row), axis=1)

C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_7352\4091229020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kadr['ans'] = kadr.apply(lambda row: fil(row), axis=1)


In [39]:
pd.merge(kadr[['СНИЛС', 'ДР','ans']], zp[zp.stv < 1][['org', 'snils']], how='right', left_on='СНИЛС', right_on='snils')[['org', 'snils', 'ДР', 'ans']].drop_duplicates('snils')

,org,snils,ДР,ans
0,ГБПОУ СКИСиГ,019-646-991 96,08.01.1964,0.0
1,ГБПОУ СКИСиГ,093-089-262 88,19.03.1985,0.0
2,ГБОУ Школа № 1502,001-427-203 78,02.10.1969,0.0
4,ГБОУ Школа № 1502,001-481-425 96,30.05.1960,0.0
5,ГБОУ Школа № 1502,001-620-488 89,13.02.1973,0.0
...,...,...,...,...
77392,ГБОУ Школа «Содружество»,016-022-590 00,17.03.1968,0.0
77393,ГБОУ Школа «Содружество»,051-174-521 23,07.05.1957,0.0
77394,ГБОУ Школа «Содружество»,072-964-459 00,17.07.1971,0.0
77395,ГБОУ Школа «Содружество»,119-897-619 28,09.03.1986,0.0


In [40]:
print_df(pd.merge(kadr[['СНИЛС', 'ДР','ans']], zp[zp.stv < 1][['org', 'snils']], how='right', left_on='СНИЛС', right_on='snils')[['org', 'snils', 'ДР', 'ans']].drop_duplicates('snils'), 'cnt_bd')